# Importar las librerias

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '....'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
import talib as ta
from backtesting import Strategy, Backtest

from utils import walk_forward
from utils import plot_full_equity_curve
from utils import get_wfo_stats
import numpy as np
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)

# Obtener los datos

In [3]:

# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish connection to MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2014, 3, 1, tzinfo=timezone)
utc_to = datetime(2024, 9, 1, tzinfo=timezone)
rates = mt5.copy_rates_range("EURUSDm", mt5.TIMEFRAME_D1, utc_from, utc_to)
 
mt5.shutdown()
 
df = pd.DataFrame(rates)

df['time'] = pd.to_datetime(df['time'], unit='s')
df = df.rename(columns={
    'time':'Date',
    'open':'Open',
    'high':'High',
    'low':'Low',
    'close':'Close',
})

df = df.set_index('Date')
df

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


,Open,High,Low,Close,tick_volume,spread,real_volume
Date,,,,,,,
2014-03-02,1.37803,1.37870,1.37529,1.37732,4650,0,0
2014-03-03,1.37736,1.37922,1.37254,1.37345,60747,0,0
2014-03-04,1.37346,1.37808,1.37200,1.37410,58371,0,0
2014-03-05,1.37402,1.37479,1.37068,1.37307,50682,0,0
2014-03-06,1.37304,1.38724,1.37208,1.38624,71416,0,0
...,...,...,...,...,...,...,...
2024-08-27,1.11641,1.11900,1.11494,1.11766,24552,8,0
2024-08-28,1.11767,1.11797,1.11046,1.11241,28228,8,0
2024-08-29,1.11239,1.11394,1.10553,1.10797,35597,7,0


# WFO

In [4]:
def diff_pips(price1, price2, pip_value, absolute=True):
    if absolute:
        difference = abs(price1 - price2)
    else:
        difference = price1 - price2
    pips = difference / pip_value
    
    return pips


class ShortMomentum(Strategy):
    risk = 1
    atr_multiplier = 1.5
    risk_reward = 1
    pip_value = 0.01
    bars_in_position = 0
    max_hold_period = 5
    
    def init(self):
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close )
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=100)
        
    def next(self):
        
        if self.position:
            self.bars_in_position += 1
            
            if self.bars_in_position >= self.max_hold_period:
                self.position.close()
                self.bars_in_position = 0
        
        else:
            if self.adx[-1] < 25:
                return
            
            go_long = self.data.Close[-1] > self.sma[-1]
            
            if go_long and self.data.Close[-1] > self.data.Close[-2]:
                
                sl = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                tp = self.data.Close[-1] + self.atr_multiplier * self.atr[-1] * self.risk_reward
                
                sl_pips = diff_pips(self.data.Close[-1], sl, pip_value=self.pip_value, absolute=True)
                account_currency_risk = self.equity * (self.risk / 100)
                units = round(account_currency_risk / (self.pip_value * sl_pips))
                
                self.buy(
                    size=units, 
                    sl=sl,
                    tp=tp
                )
            
            if not go_long and self.data.Close[-1] < self.data.Close[-2]:
                
                sl = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                tp = self.data.Close[-1] - self.atr_multiplier * self.atr[-1] * self.risk_reward
                
                sl_pips = diff_pips(self.data.Close[-1], sl, pip_value=self.pip_value, absolute=True)
                account_currency_risk = self.equity * (self.risk / 100)
                units = round(account_currency_risk / (self.pip_value * sl_pips))
                
                self.sell(
                    size=units, 
                    sl=sl,
                    tp=tp
                )

In [5]:
def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    
lookback_bars = 1500
validation_bars = 300
warmup_bars = 200

params = {
    'atr_multiplier' : np.arange(2, 6, 0.5).tolist(),
    'risk_reward' : np.arange(0.4, 5, 0.2).tolist(),
    'maximize': optim_func_2
}

stats = walk_forward(
    ShortMomentum,
    df, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=7e-4, 
    margin=1/30, 
    cash=100_000,
    verbose=True
)

plot_full_equity_curve(
    stats._equity
)



print(stats)

train from 2014-03-02 00:00:00 to 2019-09-30 00:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2019-10-01 00:00:00 to 2020-09-16 00:00:00
{'atr_multiplier': 2.5, 'risk_reward': 0.4}
equity final: 96625.80746046596
train from 2015-02-15 00:00:00 to 2020-09-16 00:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2020-09-17 00:00:00 to 2021-09-03 00:00:00
{'atr_multiplier': 2.0, 'risk_reward': 1.6000000000000005}
equity final: 95200.68441620571
train from 2016-02-02 00:00:00 to 2021-09-03 00:00:00


...

  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2022-08-21 00:00:00 to 2023-08-04 00:00:00
{'atr_multiplier': 5.0, 'risk_reward': 0.6000000000000001}
equity final: 92649.7870494267
train from 2018-02-22 00:00:00 to 2023-08-04 00:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2023-08-06 00:00:00 to 2024-07-22 00:00:00
{'atr_multiplier': 3.5, 'risk_reward': 0.4}
equity final: 90454.86700290265


Start                     2014-03-02 00:00:00
End                       2024-09-01 00:00:00
Duration                   3836 days 00:00:00
Exposure Time [%]                   11.499227
Equity Final [$]                 90454.867003
Equity Peak [$]                 100697.177314
Return [%]                          -10.14285
Buy & Hold Return [%]              -19.813841
Return (Ann.) [%]                   -1.780698
Volatility (Ann.) [%]                2.183965
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -10.26257
Avg. Drawdown [%]                   -5.131664
Max. Drawdown Duration     1790 days 00:00:00
Avg. Drawdown Duration      899 days 00:00:00
# Trades                                  175
Win Rate [%]                             48.0
Best Trade [%]                        2.00124
Worst Trade [%]                     -2.499047
Avg. Trade [%]                    